# NormLIME

NormLIME method aggregates local models into global and class-specific interpretations. It is effective at recognizing important features. In this notebook, we use NormLIME method discover the words that contribute the most to positive and negative sentiment predictions.

In [1]:
import sys

sys.path.insert(0, "../..")
sys.path.insert(0, "../../../")

# Load model parameters

We provide pre-trained model parameters. if users want to understand the training process, they can refer to this [page](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples/text_classification/rnn).

In [2]:
import numpy as np
import paddle
import paddlenlp
from paddlenlp.data import Vocab
from paddlenlp.data import JiebaTokenizer

!wget --no-check-certificate -c https://trustai.bj.bcebos.com/chnsenticorp-bilstm.tar
!tar -xvf chnsenticorp-bilstm.tar -C ../../assets
!rm ./chnsenticorp-bilstm.tar
PARAMS_PATH = "../../assets/chnsenticorp-bilstm/final.pdparams"
VOCAB_PATH = "../../assets/chnsenticorp-bilstm/bilstm_word_dict.txt"

# init config
vocab = Vocab.from_json(VOCAB_PATH)
tokenizer = JiebaTokenizer(vocab)
label_map = {0: 'negative', 1: 'positive'}
vocab_size = len(vocab)
num_classes = len(label_map)
pad_token_id = vocab.to_indices('[PAD]')


--2022-04-07 20:58:33--  https://trustai.bj.bcebos.com/chnsenticorp-bilstm.tar
Resolving trustai.bj.bcebos.com (trustai.bj.bcebos.com)... 10.70.0.165
Connecting to trustai.bj.bcebos.com (trustai.bj.bcebos.com)|10.70.0.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13690880 (13M) [application/x-tar]
Saving to: ‘chnsenticorp-bilstm.tar’

100%[======================================>] 13,690,880  62.1MB/s   in 0.2s   

2022-04-07 20:58:33 (62.1 MB/s) - ‘chnsenticorp-bilstm.tar’ saved [13690880/13690880]

chnsenticorp-bilstm/
chnsenticorp-bilstm/final.pdparams
chnsenticorp-bilstm/bilstm_word_dict.txt


In [3]:
from assets.utils import LSTMModel

# init model
model = LSTMModel(vocab_size, num_classes, direction='bidirect', padding_idx=pad_token_id)
state_dict = paddle.load(PARAMS_PATH)
model.set_dict(state_dict)

W0407 20:58:34.161798 24117 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.4, Runtime API Version: 10.2
W0407 20:58:34.170097 24117 device_context.cc:465] device: 0, cuDNN Version: 8.2.


Define a preprocessing function that takes in a raw string and outputs the model inputs that can be fed into paddle_model.

In this case, the raw string is splitted and mapped to word ids. texts is a list of lists, where each list contains a sequence of padded word ids. seq_lens is a list that contains the sequence length of each unpadded word ids in texts.

Since the input data is a single raw string. Both texts and seq_lens has length 1.

In [4]:
from functools import partial

def preprocess_fn(text, tokenizer):
    ids = tokenizer.encode(text)
    texts = paddle.to_tensor([ids])
    seq_lens = paddle.to_tensor(len(ids))
    return texts, seq_lens
preprocess_fn = partial(preprocess_fn, tokenizer=tokenizer)

We use the first 1200 samples in the training set as our data.


In [5]:
from paddlenlp.datasets import load_dataset

DATASET_NAME = 'chnsenticorp'
train_ds = load_dataset(DATASET_NAME, splits=["train"])
data = [d['text'] for d in list(train_ds)[:1200]]

# NormLIMEInterpreter



In [6]:
from trustai.interpretation.token_level import NormLIMEInterpreter

normlime = NormLIMEInterpreter(model,
                               preprocess_fn,
                               unk_id=vocab.to_indices('[UNK]'),
                               pad_id=vocab.to_indices('[PAD]'),
                               batch_size=50)
result = normlime(data,
                  num_samples=500,
                  temp_data_file='../../assets/all_lime_weights.npz',
                  save_path="../../assets/normlime_weights.npy")


 91%|█████████▏| 1096/1200 [01:31<00:09, 10.50it/s]'语言空洞，情节太离谱，没有感觉到震撼，只觉得这个人太倒霉了，也没有什么可以发掘的内涵，觉得整本书写得精彩点的部分的可能就是序了' has been computed before. Check it if this is NOT expected.
'海景花园是我所有住过的５星酒店中服务最好的一家，另外他们的粤式餐厅非常地道，并且价格合理，如果去青岛还会再选它。 Fantastic 5 star hotel, good location and environment, with best service and very nice staff, strongly recommend. The Cantonese restaurant is one of the best, worth to try!!! 补充点评 2008年7月29日 ： &#32431;&#20013;&#26041;&#31649;&#29702;&#65292;&#38750;&#24120;&#26834;&#65281;' has been computed before. Check it if this is NOT expected.
100%|██████████| 1200/1200 [01:42<00:00, 11.69it/s]


In [7]:
import pandas as pd

id2word = vocab.idx_to_token

# Print interpret result
In the cells below, we print the words with top 20 largest weights for positive and negative sentiments. Only words that appear at least 5 times are included.

In [8]:
# Positive
temp = {id2word[wid]: result.attributions[1][wid] for wid in result.attributions[1]}
W = [(word, weight[0], weight[1]) for word, weight in temp.items() if weight[1] >= 5]
pd.DataFrame(data=sorted(W, key=lambda x: -x[1])[:20], columns=['word', 'weight', 'frequency'])


,word,weight,frequency
0,改进,0.029098,5
1,电子版,0.022340,5
2,很值,0.012091,8
3,定,0.010631,5
4,流畅,0.010191,11
5,小巧,0.009392,10
6,一句,0.008710,5
7,舒适,0.007505,9
8,亮,0.007260,21
9,大方,0.006879,5


In [9]:
# Negative
temp = {id2word[wid]: result.attributions[0][wid] for wid in result.attributions[0]}
W = [(word, weight[0], weight[1]) for word, weight in temp.items() if weight[1] >= 5]
pd.DataFrame(data=sorted(W, key=lambda x: -x[1])[:20], columns=['word', 'weight', 'frequency'])

,word,weight,frequency
0,于丹,0.065560,7
1,声卡,0.059569,6
2,没收,0.057041,8
3,没到,0.038130,5
4,要是,0.037119,9
5,不适,0.036320,6
6,重,0.035883,10
7,不爽,0.035794,5
8,失望,0.031051,23
9,不然,0.030569,7
